In [1]:
import torch
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader, Dataset

In [6]:
# Charger les données
import pandas as pd
data = pd.read_csv("datasets\hespress_proceced")

In [7]:
data.head()

,post_content,note,processed_text
0,تسلح يونس السكوري، وزير الإدماج الاقتصادي والم...,3.392897,تسلح يونس سكور زير ادماج اقتصاد مقاول صغرى تشغ...
1,أعادت واقعة “الاعتداء” الذي طال مسؤولا روسيا ف...,3.580410,اعاد واقع “ اعتداء ” طال مسوول روس دار بيضاء ن...
2,على بعد أسابيع من احتفالات رأس السنة تتجدد الم...,7.154203,اسابيع احتفال راس السن تتجدد مخاوف ارتفاع حال ...
3,كشف وزير العدل، عبد اللطيف وهبي، وجود محاولات ...,1.114053,كشف زير عدل عبد لطيف وهب جود محاول تخوض وزار ل...
4,نعت “مؤسسة دار الحديث الحسنية” التابعة لجامعة ...,1.107434,نعت “ موسس دار حديث حسن ” تابع جامع قرو اكاديم...


In [25]:
# Tokenisation simple
data['tokens'] = data['processed_text'].apply(lambda x: x.split())

# Construire un vocabulaire
vocab = build_vocab_from_iterator(data['tokens'], specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

# Transformer les textes en séquences d'entiers
data['encoded'] = data['tokens'].apply(lambda x: [vocab[token] for token in x])

# Remplir ou tronquer les séquences pour une longueur fixe
MAX_LEN = 50
data['padded'] = data['encoded'].apply(lambda x: x[:MAX_LEN] + [0] * max(0, MAX_LEN - len(x)))

In [26]:
class TextDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = torch.tensor(self.data.iloc[idx]['padded'], dtype=torch.long)
        label = torch.tensor(self.data.iloc[idx]['note'], dtype=torch.float)
        return text, label

dataset = TextDataset(data)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)


In [13]:
import torch.nn as nn

class SequenceModel(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, model_type="RNN"):
        super(SequenceModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.model_type = model_type
        if model_type == "RNN":
            self.rnn = nn.RNN(embed_size, hidden_size, batch_first=True)
        elif model_type == "BiRNN":
            self.rnn = nn.RNN(embed_size, hidden_size, batch_first=True, bidirectional=True)
        elif model_type == "GRU":
            self.rnn = nn.GRU(embed_size, hidden_size, batch_first=True)
        elif model_type == "LSTM":
            self.rnn = nn.LSTM(embed_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size * (2 if "Bi" in model_type else 1), 1)

    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.rnn(x)
        x = x[:, -1, :]  # Dernière sortie temporelle
        x = self.fc(x)
        return x


In [27]:
import torch.optim as optim
from torch.nn import MSELoss
from tqdm import tqdm

# Initialiser le modèle
vocab_size = len(vocab)
embed_size = 100
hidden_size = 64
model_type = "LSTM"  # Changez pour RNN, BiRNN, GRU, ou LSTM

model = SequenceModel(vocab_size, embed_size, hidden_size, model_type)
criterion = MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Boucle d'entraînement
for epoch in range(100):
    model.train()
    total_loss = 0
    for texts, scores in tqdm(dataloader, desc=f"the epoch { epoch}/ 100"):
        optimizer.zero_grad()
        outputs = model(texts)
        loss = criterion(outputs.squeeze(), scores)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch + 1}, Loss: {total_loss / len(dataloader):.4f}")


the epoch 0/ 100: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 62.67it/s]


Epoch 1, Loss: 24.4899


the epoch 1/ 100: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 29.49it/s]


Epoch 2, Loss: 23.9633


the epoch 2/ 100: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 34.57it/s]


...

Epoch 98, Loss: 0.2939


the epoch 98/ 100: 100%|█████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 52.78it/s]


Epoch 99, Loss: 0.2885


the epoch 99/ 100: 100%|█████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 35.81it/s]

Epoch 100, Loss: 0.2830


In [28]:
from sklearn.metrics import mean_squared_error

model.eval()
predictions = []
actuals = []
with torch.no_grad():
    for texts, scores in dataloader:
        outputs = model(texts)
        predictions.extend(outputs.squeeze().tolist())
        actuals.extend(scores.tolist())

mse = mean_squared_error(actuals, predictions)
print(f"Mean Squared Error: {mse:.4f}")

Mean Squared Error: 0.2774
